In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Input
from tensorflow.keras.optimizers import Adam
import sys
sys.path.append('../..')
from helpers import lookups

In [3]:
target = 'tmed'
seed = 42

In [9]:
df = pd.read_csv('../../data/historical/2024_clean_scaled.csv', parse_dates=['fecha'])
df = df.set_index('fecha')
df.head()

,tmed,prec,tmin,tmax,hr_max,hr_min,hr_media,velmedia,racha,pres_max,pres_min,sol,year,fecha_sin,fecha_cos,dir_sin,dir_cos
fecha,,,,,,,,,,,,,,,,,
2024-01-01,0.4564,0.0,0.4712,0.4309,0.5794,0.1313,0.5510,0.1450,0.2027,0.6297,0.6292,0.3986,0.0,0.5086,1.0,0.9528,0.1623
2024-01-01,0.6349,0.0,0.6482,0.5957,0.7850,0.4646,0.6531,0.0706,0.0808,0.8065,0.8065,0.3392,0.0,0.5086,1.0,0.8872,0.7463
2024-01-01,0.4481,0.0,0.4093,0.4681,0.8692,0.6364,0.7857,0.0409,0.0808,0.9287,0.9277,0.3892,0.0,0.5086,1.0,0.9185,0.0936
2024-01-01,0.6079,0.0,0.6018,0.5851,0.7383,0.5253,0.7347,0.0818,0.1098,0.9463,0.9460,0.4892,0.0,0.5086,1.0,0.7522,0.8823
2024-01-01,0.3776,0.0,0.3252,0.4113,0.8692,0.5758,0.7959,0.0632,0.0808,0.7926,0.7948,0.3108,0.0,0.5086,1.0,0.9726,0.5751


In [10]:
def create_dataset(data, target_idx, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), target_idx]) 
        y.append(data[i + time_step, target_idx]) 
    return np.array(X), np.array(y)

In [11]:
target_idx = df.columns.get_loc(target)
target_idx

0

In [13]:
time_step = 100 
X, y = create_dataset(df.values, target_idx, time_step)
X = X.reshape(X.shape[0], X.shape[1], 1)

In [14]:
X.shape, y.shape

((323443, 100, 1), (323443,))

In [15]:
def create_gru():
    model = Sequential()
    model.add(Input(shape=(X.shape[1], X.shape[2])))
    model.add(GRU(units=50, return_sequences=True))
    model.add(GRU(units=50))
    model.add(Dense(units=1))
    
    return model

In [17]:
model = create_gru()
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

In [19]:
model.fit(X, y, epochs=1, batch_size=1028)

315/315 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - loss: 0.0076
